# Manisha Pednekar NLP Homework 8

<a id='BacktoTop'></a>
## <font color=blue> Contents </font>
- [Question 1: Load one sentiment vocabulary and Write a function for the sentiment analyzer](#section_ID1)
- [Question 2: compute the average, median, high, and low sentiment scores for each cluster](#section_ID2)

In [5]:
import pandas as pd
import numpy as np
import text_normalizer as tn

In [53]:
import os;
print(os.getenv("CONDA_DEFAULT_ENV")) # I am getting Key error for Ben's code: os.environ['CONDA_DEFAULT_ENV']
import platform; print(platform.platform())
import sys; print("Python", sys.version)
print("Pandas", pd.__version__)
print("NumPy", np.__version__)
import re; print("Re", re.__version__)
import nltk; print( "nltk", nltk.__version__)

None
Windows-10-10.0.17763-SP0
Python 3.6.5 |Anaconda, Inc.| (default, Mar 29 2018, 13:32:41) [MSC v.1900 64 bit (AMD64)]
Pandas 0.25.0
NumPy 1.16.4
Re 2.2.1
nltk 3.4.4


<a id="section_ID1"></a>
<font color=red> Perform a vocabulary-based sentiment analysis of the movie reviews you used in homework 5 and homework 7, by doing the following: <br>
1.	In Python, load one of the sentiment vocabularies referenced in the textbook, and run the sentiment analyzer as explained in the corresponding reference. Add words to the sentiment vocabulary, if you think you need to, to better fit your particular text collection.</font> <br>
I collected movie reviews on four different movies with 25 reviews per movie of genre "Animation", thus 100 reviews total. The movies I chose to create this movie review dataset are Cars 3, Toy Story 4, The Grinch and Aladdin. I created a .csv file named “clusterDf” in HW7 from the output of k-means clustering. It contains the following 7 columns: **Movie Name, User rating, review title, review text, Full text Combining title and text, Label based on user rating (positive/ negative) and Cluster# (0 to 3).** Here I am loading this .csv file from the movie reviews clustering HW 7 assignment.

I have used Dipanjan Sarkar's code from this GitHub website https://github.com/dipanjanS/practical-machine-learning-with-python/blob/master/notebooks/Ch07_Analyzing_Movie_Reviews_Sentiment/unsupervised_sentiment_analysis.py. I tried Sarkar books code assigned for this course, but it had a lot of legacy code (Python 2 code) to repair. Thus, I used Sarkar's book called "Practical machine learning with Python". This code needs 2 supporting python files (text_normalizer.py, contractions.py) to run.

During testing of the sentiment analysis funtion code, I found that one the review is so small (review text: "Why") that it was not fitting any of the POS tag from the funtion code, ultimately it was resulting as division by zero error. Thus I deleted that review record which row# 11 from the dataframe.

In [54]:
np.set_printoptions(precision=2, linewidth=80)
df = pd.read_csv("clusterDf.csv")
reviews = np.array(df['fullText'])

In [7]:
from nltk.corpus import sentiwordnet as swn

In [8]:
# Testing sentiwordnet library
awesome = list(swn.senti_synsets('awesome', 'a'))[0]
print('Positive Polarity Score:', awesome.pos_score())
print('Negative Polarity Score:', awesome.neg_score())
print('Objective Score:', awesome.obj_score())

Positive Polarity Score: 0.875
Negative Polarity Score: 0.125
Objective Score: 0.0


[Back to the top](#BacktoTop)

In [44]:
def analyze_sentiment_sentiwordnet_lexicon(review,
                                           verbose=False):

    # tokenize and POS tag text tokens
    tagged_text = [(token.text, token.tag_) for token in tn.nlp(review)]
    pos_score = neg_score = token_count = obj_score = 0
    # get wordnet synsets based on POS tags
    # get sentiment scores if synsets are found
    for word, tag in tagged_text:
        ss_set = None
        if 'NN' in tag and list(swn.senti_synsets(word, 'n')):
            ss_set = list(swn.senti_synsets(word, 'n'))[0]
        elif 'VB' in tag and list(swn.senti_synsets(word, 'v')):
            ss_set = list(swn.senti_synsets(word, 'v'))[0]
        elif 'JJ' in tag and list(swn.senti_synsets(word, 'a')):
            ss_set = list(swn.senti_synsets(word, 'a'))[0]
        elif 'RB' in tag and list(swn.senti_synsets(word, 'r')):
            ss_set = list(swn.senti_synsets(word, 'r'))[0]
        # if senti-synset is found        
        if ss_set:
            # add scores for all found synsets
            pos_score += ss_set.pos_score()
            neg_score += ss_set.neg_score()
            obj_score += ss_set.obj_score()
            token_count += 1
    
    # aggregate final scores
    final_score = pos_score - neg_score
    norm_final_score = round(float(final_score) / token_count, 2)
    final_sentiment = 'positive' if norm_final_score >= 0 else 'negative'
    if verbose:
        norm_obj_score = round(float(obj_score) / token_count, 2)
        norm_pos_score = round(float(pos_score) / token_count, 2)
        norm_neg_score = round(float(neg_score) / token_count, 2)
        # to display results in a nice table
        sentiment_frame = pd.DataFrame([[final_sentiment, norm_obj_score, norm_pos_score, 
                                         norm_neg_score, norm_final_score]],
                                       columns=pd.MultiIndex(levels=[['SENTIMENT STATS:'], 
                                                             ['Predicted Sentiment', 'Objectivity',
                                                              'Positive', 'Negative', 'Overall']], 
                                                             labels=[[0,0,0,0,0],[0,1,2,3,4]]))
        print(sentiment_frame)
        
    return norm_final_score

In [45]:
df2 = df.copy()
df2 = df2.drop(df2.index[[11]])#had to delete one record since the review is one word "Why" and does not fit in any of the POS tag
#df2 = df2.loc[20:,:]
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 99 entries, 0 to 99
Data columns (total 7 columns):
Label        94 non-null object
MovieName    99 non-null object
rRating      94 non-null float64
rText        99 non-null object
rTitle       99 non-null object
fullText     99 non-null object
cluster      99 non-null int64
dtypes: float64(1), int64(1), object(5)
memory usage: 6.2+ KB


[Back to the top](#BacktoTop)

In [46]:
swnScoreList = []

for review in df2['fullText'] :
    swnScore = analyze_sentiment_sentiwordnet_lexicon(review, verbose = True)
    swnScoreList.append(swnScore)

C:\Users\Sohum Pednekar\projNlp\lib\site-packages\ipykernel_launcher.py:41: FutureWarning: the 'labels' keyword is deprecated, use 'codes' instead


     SENTIMENT STATS:                                      
  Predicted Sentiment Objectivity Positive Negative Overall
0            negative         0.6     0.19     0.21   -0.02
     SENTIMENT STATS:                                      
  Predicted Sentiment Objectivity Positive Negative Overall
0            positive        0.56     0.44      0.0    0.44
     SENTIMENT STATS:                                      
  Predicted Sentiment Objectivity Positive Negative Overall
0            negative        0.97      0.0     0.03   -0.03
     SENTIMENT STATS:                                      
  Predicted Sentiment Objectivity Positive Negative Overall
0            negative        0.38     0.12      0.5   -0.38
     SENTIMENT STATS:                                      
  Predicted Sentiment Objectivity Positive Negative Overall
0            negative         0.5      0.0      0.5    -0.5
     SENTIMENT STATS:                                      
  Predicted Sentiment Objectivity Positi

0            negative        0.88     0.05     0.08   -0.03
     SENTIMENT STATS:                                      
  Predicted Sentiment Objectivity Positive Negative Overall
0            positive        0.85     0.09     0.06    0.03
     SENTIMENT STATS:                                      
  Predicted Sentiment Objectivity Positive Negative Overall
0            positive        0.88      0.1     0.02    0.08
     SENTIMENT STATS:                                      
  Predicted Sentiment Objectivity Positive Negative Overall
0            positive        0.88     0.08     0.05    0.03
     SENTIMENT STATS:                                      
  Predicted Sentiment Objectivity Positive Negative Overall
0            positive        0.75     0.13     0.12    0.01
     SENTIMENT STATS:                                      
  Predicted Sentiment Objectivity Positive Negative Overall
0            negative        0.88      0.0     0.12   -0.12
     SENTIMENT STATS:                   

0            positive        0.72      0.2     0.08    0.12
     SENTIMENT STATS:                                      
  Predicted Sentiment Objectivity Positive Negative Overall
0            positive        0.84      0.1     0.06    0.04
     SENTIMENT STATS:                                      
  Predicted Sentiment Objectivity Positive Negative Overall
0            positive        0.82     0.16     0.01    0.15
     SENTIMENT STATS:                                      
  Predicted Sentiment Objectivity Positive Negative Overall
0            positive        0.89     0.07     0.04    0.02
     SENTIMENT STATS:                                      
  Predicted Sentiment Objectivity Positive Negative Overall
0            positive        0.74     0.17     0.09    0.08
     SENTIMENT STATS:                                      
  Predicted Sentiment Objectivity Positive Negative Overall
0            positive        0.84     0.09     0.07    0.02
     SENTIMENT STATS:                   

In [47]:
len(swnScoreList)

99

In [48]:
df2['swnScore'] = swnScoreList

In [50]:
swnScoreList[:8]

[-0.02, 0.44, -0.03, -0.38, -0.5, 0.02, -0.17, 0.05]

<a id="section_ID2"></a>
<font color=red> 2.	For each of the clusters you created in homework 7, compute the average, median, high, and low sentiment scores for each cluster. Explain whether you think this reveals anything interesting about the clusters.
</font>
### Sentiment Analysis
I used “sentiwordnet” lexicon library from nltk package for Sentiment Analysis of these reviews. For each cluster, the means and medians of sentiments scores are aligned with the corresponding *means and medians* of review User ratings. The means of the sentiment scores are more aligned means of the ratings than min, median or max measures. Thus, we can conclude that “sentiwordnet" lexicon library can be used successfully for this set of reviews despite the very small size of the corpus used for sentiment analysis task.


In [52]:
# code taken from this website: https://jakevdp.github.io/PythonDataScienceHandbook/03.08-aggregation-and-grouping.html
df2.groupby('cluster').aggregate({'rRating' :[min, 'mean', 'median', max], 'swnScore': [min, 'mean', 'median',max]})

rRating                        swnScore                       
            min      mean median   max      min      mean median   max
cluster                                                               
0           1.0  6.428571    8.0  10.0    -0.05  0.045000  0.040  0.19
1           7.0  8.882353    9.0  10.0    -0.05  0.049524  0.040  0.17
2           7.0  9.750000   10.0  10.0    -0.03  0.067500  0.075  0.12
3           1.0  4.413793    1.0  10.0    -0.62 -0.018000 -0.010  0.44

[Back to the top](#BacktoTop)